$\Large\color{blue}{\text{PyGMTSAR Crete Earthquake M5.8 on Sep 27, 2021 Co-Seismic Interferogram}}$
$\large\color{blue}{\text{Compare the results to Centre of EO Research & Satellite Remote Sensing, Greece report}}$

### See and more PyGMTSAR notebooks on GitHub: [PyGMTSAR](https://github.com/mobigroup/gmtsar)

It works on

* MacOS Monterey (Apple Silicon) and BigSur (Intel) (Python 3.9) - please pre-install system dependencies (maybe using HomeBrew),

* Google Cloud VM and Notebooks on Debian 10 (Python 3.7), use the [Google Cloud Debian 10 init script](https://github.com/mobigroup/gmtsar/blob/master/gmtsar/sh/GMTSAR.install.debian10.sh) and [Google Cloud Debian 10 VM creation script](https://github.com/mobigroup/gmtsar/blob/master/gmtsar/sh/GMTSAR.gcloud_create_debian10.sh)

* $\color{red}{\text{Google Colab (Python 3.7) - you will be asked to re-run the notebook once due to "system crash" by desing}}$
$\color{red}{\text{Note: to open all notebook cells select menu "View" -> "Expand Sections"}}$

### PyGMTSAR is my free-time Open Source project.

That's a bit curious how the project was started a year ago. I develop geophysical inversion methods and processing software for many years using my fundamental physics and mathematics background. Satellite interferometry is the key point to validate my inversion models and I found the same problem as you too that the existing interferometry packages usage is a pain. There is no interactive processing with multiprocessing and even a progressbar and ability to view and change every step code and validate the results. Also, many used algorithms are too outdated and produce terrible results like to tension surfaces in GMT which is used widely in GMTSAR (hmm, how about to control a smothness of derivative? Tension surfaces were invented when all the Earths computers where less powerfull than your smarthone today. If you are interested I shared the examples in GMTSAR bug tracker). Anyway, I found GMT mathematics is really crazy and the developers replace one incorrect algorithm by another and back often as we see in the codes and the changelog). That was enough reason to check all the used algorithms and replace these by modern and correct ones. By this way, I use only GMTSAR C codes with my patches to fix some errors and allow interoperability with Python wrappers plus my own codes around them. GMTSAR codes are fine and without crazy GMT codes work better and the processing is much faster. Alright, I spent one month to make the initial PyGMTSAR realization and it works. Recently, I returned to the project to add some more sophisticated features like to scenes and subswathes stitching. I'm going to share some of my geological exprorations and seismic models as live examples on Google Colab as soon as it will be possible to do. How lineaments and ore zones are related to interferograms? How gas and oil deposits are related to surface movements on interferometry displacement maps? I have the answer and I work on the tools to model and visualize them.

You'd find my theoretical models and processing codes foir geophisical inversions in Github repository https://github.com/mobigroup/gis-snippets and tools for the 4D results vizualization in https://github.com/mobigroup/ParaView-plugins

Ah yes, a little bit about me. I have STEM master's degree in radio physics and in 2004 I was awarded first prize of the All-Russian Physics competition for significant results in Inverse modeling for non-linear optics and holography, also applicable for Inverse Modeling of Gravity, Magnetic, and Thermal fields. In addition to my fundamental science knowledge, I’m world class data scientist and software developer with 20 years experience in science and industrial development. I have worked on government contracts and universities projects and on projects for LG Corp, Google Inc, etc. You are able to find some of my software and results on LinkedIn and GitHub and Upwork. By the way, I left Russia many years ago and I work remotely for about 20 years.

### To order some research, development and support see my profile on freelance platform [Upwork](https://www.upwork.com/freelancers/~01e65e8e7221758623)

### @ Alexey Pechnikov, August, 2022

[Geological models on YouTube channel](https://www.youtube.com/channel/UCSEeXKAn9f_bDiTjT6l87Lg)

[Augmented Reality (AR) Geological Models](https://mobigroup.github.io/ParaView-Blender-AR/)

[GitHub repositories](https://github.com/mobigroup)

[English posts and articles on LinkedIn](https://www.linkedin.com/in/alexey-pechnikov/)

[Russian articles on Habr](https://habr.com/ru/users/N-Cube/posts/)

$\large\color{blue}{\text{Hint: Use menu Cell} \to \text{Run All or Runtime} \to \text{Complete All or Runtime} \to \text{Run All}}$
$\large\color{blue}{\text{(depending of your localization settings) to execute the entire notebook}}$

## Load Modules to Check Environment

In [ ]:
import platform, sys, os

## Debian 10 and Google Colab GMTSAR Installation

### On Google Cloud AI Notebooks: check root access

On Google Cloud AI Notebooks sometimes we have an issue when "sudo" requires a password. In this case drop the instance and create a new one and - that's important - wait 5-10 minutes before connect to it using link "OPEN JUPYTERLAB"

In [ ]:
if platform.system() == 'Linux':
    !sudo date

### Install https://github.com/mobigroup/gmtsar

I make lots of changes on GMTSAR C-coded tools and some of them are not merged to the upstream GMTSAR yet because all the patches need to be validated and discussed before. Also, my Python extensions are provided in my GMTSAR fork only. I hope in the future to provide a standalone python packager as wrapper around upstream GMTSAR but there is a long way to it.

In [ ]:
if platform.system() == 'Linux':
    count = !ls /usr/local | grep GMTSAR | wc -l
    if count == ['0']:
        !apt install -y csh autoconf gfortran \
            libtiff5-dev libhdf5-dev liblapack-dev libgmt-dev gmt-dcw gmt-gshhg gmt > /dev/null
        !cd /usr/local && git clone --branch master https://github.com/mobigroup/gmtsar GMTSAR > /dev/null
        !cd /usr/local/GMTSAR && autoconf > /dev/null
        !cd /usr/local/GMTSAR && ./configure --with-orbits-dir=/tmp > /dev/null
        !cd /usr/local/GMTSAR && make 1>/dev/null 2>/dev/null
        !cd /usr/local/GMTSAR && make install >/dev/null

## Define ENV Variables for Jupyter Instance

In [ ]:
# use default GMTSAR installation path
GMTSAR = '/usr/local/GMTSAR'
PATH = os.environ['PATH']

if PATH.find('GMTSAR') == -1:
    PATH = os.environ['PATH'] + f':{GMTSAR}/bin/'
    %env PATH {PATH}
    %env GMTSAR {GMTSAR}

## Install Python Modules

Maybe you need to restart your notebook, follow the instructions printing below.

The installation takes a long time on fresh Debian 10 and a short time on Google Colab

In [ ]:
!{sys.executable} --version

In [ ]:
if platform.system() == 'Linux':
    !{sys.executable} -m pip install cartopy==0.19.0.post1 1>/dev/null 2>/dev/null
    !{sys.executable} -m pip install xarray==0.19.0        1>/dev/null 2>/dev/null
    !{sys.executable} -m pip install scipy==1.7.1          1>/dev/null 2>/dev/null

In [ ]:
if platform.system() == 'Linux':
    !{sys.executable} -m pip install \
        h5py netcdf4 h5netcdf \
        rasterio rioxarray numpy \
        scikit-image scipy sklearn \
        xarray dask distributed zarr nc-time-axis \
        pandas geopandas \
        sentineleof elevation \
        matplotlib seaborn geoviews hvplot datashader bokeh \
        xmltodict joblib tqdm 1>/dev/null 2>/dev/null

$\large\color{red}{\text{Attention: On Google Colab we need to restart kernel once when modules installed}}$

$\large\color{blue}{\text{Hint: Use menu Cell} \to \text{Run All or Runtime} \to \text{Complete All or Runtime} \to \text{Run All}}$
$\large\color{blue}{\text{(depending of your localization settings) to execute the entire notebook}}$

In [ ]:
if platform.system() == 'Linux':
    import xarray
    import time
    print (xarray.__version__)
    if xarray.__version__ != '0.19.0':
        print ("""
    ***********************************************************************************
    *
    Do not worry, runtime is stopped by design. Please run the notebook again.
    *
    ***********************************************************************************
    """)
        time.sleep(1)
        os.kill(os.getpid(), 9)

## Load and Setup Python Modules

In [ ]:
import xarray as xr
import numpy as np
import pandas as pd
# supress numpy warnings
import warnings
warnings.filterwarnings('ignore')

In [ ]:
# plotting modules
import hvplot.pandas  # noqa
import hvplot.xarray  # noqa
import holoviews as hv
pd.options.plotting.backend = 'hvplot'
from IPython.display import Image
import matplotlib.pyplot as plt
import matplotlib
%matplotlib inline

In [ ]:
# define Pandas display settings
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', None)

gstiles = hv.Tiles('https://mt1.google.com/vt/lyrs=s&x={X}&y={Y}&z={Z}', name='Google Satellite')
ottiles = hv.Tiles('https://tile.opentopomap.org/{Z}/{X}/{Y}.png', name='Open Topo')

## Load Custom Python Modules

In [ ]:
sys.path.append(os.path.join(os.environ['GMTSAR'],'gmtsar', 'py'))

from PRM import PRM
from SBAS import SBAS

## Download and Unpack RAW Sentinel-1 Scenes

$\large\color{blue}{\text{Hint: Use site https://search.asf.alaska.edu/ to search and download the scenes (registration required)}}$
$\large\color{blue}{\text{Note: Below we downloading the scenes from my server to miss the authorization step}}$

In [ ]:
# use Apple cloud share url with added file extension to it (by default it includes only filename)
# the extended url is still valid and it can be opened in web browser too
icloud_urls = {'https://www.icloud.com/iclouddrive/060EcVsMZF5o5nDoyAxHZ-JgQ#S1A_IW_SLC__1SDV_20210929T162245_20210929T162312_039899_04B89A_651D.zip',
               'https://www.icloud.com/iclouddrive/0bcQQkK2QDSiX76_y1O6zfd_Q#S1B_IW_SLC__1SDV_20210923T162204_20210923T162231_028828_0370C4_681D.zip',
               'https://www.icloud.com/iclouddrive/085_X__5ZszrqcnP--SGCH-tw#S1AB_Crete_Earthquake_vs_geObsevatory.jpg'}

if platform.system() == 'Linux':
    !apt install -y jq > /dev/null

for icloud_url in icloud_urls:
    uid, fname = icloud_url.split('/')[-1].split('#')
    if os.path.isfile(fname):
        print ('Already exists downloaded file', fname)
        continue
    print ('Downloading iCloud file', fname)
    # use iCloud API to obtain the direct link to download the file
    request = f'{{"shortGUIDs":[{{"value":"{uid}"}}]}}'
    urls = !curl -s 'https://ckdatabasews.icloud.com/database/1/com.apple.cloudkit/production/public/records/resolve' \
    --data-raw '{request}' --compressed | jq -r '.results[0].rootRecord.fields.fileContent.value.downloadURL'
    !curl -so '{fname}' '{urls[0]}'

In [ ]:
!mkdir -p data
!unzip -j -n S1A_IW_SLC__1SDV_20210929T162245_20210929T162312_039899_04B89A_651D.zip \
    '*.SAFE/*/s1?-iw3-slc-vv-*-006.xml' -d data
!unzip -j -n S1A_IW_SLC__1SDV_20210929T162245_20210929T162312_039899_04B89A_651D.zip \
    '*.SAFE/*/s1?-iw3-slc-vv-*-006.tiff' -d data

!unzip -j -n S1B_IW_SLC__1SDV_20210923T162204_20210923T162231_028828_0370C4_681D.zip \
    '*.SAFE/*/s1?-iw3-slc-vv-*-006.xml' -d data
!unzip -j -n S1B_IW_SLC__1SDV_20210923T162204_20210923T162231_028828_0370C4_681D.zip \
    '*.SAFE/*/s1?-iw3-slc-vv-*-006.tiff' -d data

## Define Processing Parameters

$\large\color{blue}{\text{Note: here are already prepared tiff and xml files in the DATADIR}}$

In [ ]:
WORKDIR      = 'raw'
DATADIR      = 'data'
CORRLIMIT    = 0.10
DEFOMAX      = 0

## Init SBAS

Search recursively for measurement (.tiff) and annotation (.xml) and orbit (.EOF) files in the DATA directory. It can be directory with full unzipped scenes (.SAFE) subdirectories or just a directory with the list of pairs of required .tiff and .xml files (maybe pre-filtered for orbit, polarization and subswath to save disk space). If orbit files and DEM are missed these will be downloaded automatically below.

In [ ]:
# and can use pre-downloaded DEM too
sbas = SBAS(DATADIR, basedir=WORKDIR)

In [ ]:
sbas.to_dataframe()

### Download Sentinel-1 Orbits

The function below downloads orbit files.
Besides, for faster processing we can automatically use pre-downloaded orbit files in data directory.

In [ ]:
# can place pre-downloaded orbit files to data directory 
sbas.download_orbits()

In [ ]:
sbas.to_dataframe()

## Reframe Scenes (Optional)

Stitch sequential scenes and crop the subswath to a smaller area for faster processing when the full area is not needed.

### Define Pair of Pins for Each Subswath (Optional)

To create a stitched and maybe cropped frame two pins should be defined. Use the plots below to define the pins coordinates. Hint: in case of an error you see the notes how to fix the issues and you are still able to map the pins using the code below (anyway the issues should be fixed to continue the calculations).

This step can be missed and for this case automatically generated pins cover the full master image.

In [ ]:
# define only one pin and crop from it and up to the top
sbas.set_pins([25.3, 35.0, None, None])

### Static Plot

In [ ]:
plt.figure(figsize=(8,4), dpi=150)
#sbas.get_dem()[::4,::4].plot.imshow(cmap='Blues_r', vmin=0)
gdf =sbas.to_dataframe()
cmap = plt.cm.get_cmap('bwr', len(gdf.index.unique()))
colors = dict([(v,cmap(k)) for k,v in enumerate(gdf.index.unique())])
gdf.reset_index().plot(color=[colors[k] for k in gdf.index], alpha=0.2, edgecolor='black', ax=plt.gca())
plt.scatter(x=sbas.get_pins()[::2], y=sbas.get_pins()[1::2], color='black', s=10)
plt.title('Sentinel1 Scenes Approximate Location', fontsize=12)
plt.show()

### Interactive Plot

Be careful because interactive plots require more RAM to be visualized

The plots below do not work on Debian 10 and Python 3.7

In [ ]:
plots = None
if platform.system() == 'Darwin':
    opts_common = {'x':'lon', 'y':'lat', 'geo':True, 'width':800, 'height':600}
    opts1 = {'tiles':gstiles, 'alpha':0.4, 'title': 'Sentinel1 Scenes Approximate Location'}
    plots = sbas.to_dataframe().reset_index().hvplot(color='date', **{**opts_common, **opts1}) \
        * pd.DataFrame({'x': sbas.get_pins()[::2], 'y': sbas.get_pins()[1::2]}).hvplot.points(y='y', x='x', c='black', geo=True)
plots

### Crop the Frame (Optional)

When we satisfied with the pins positions and the target frame we are ready to crop the frame.
Just comment the code cell below when you don't need to crop the area.

In [ ]:
sbas.reframe_parallel()

### Show the updated dataset  (Optional)

The newly created scene frames .tiff and .xml files plus orbit files .EOF can be copied for the future processing using `SBAS.backup()` function to use them instead of the original full scenes.

In [ ]:
sbas.to_dataframe()

### Download SRTM DEM

The function below downloads SRTM1 or SRTM3 DEM and converts heights to ellipsoidal model using EGM96 grid.
Besides, for faster processing we can use pre-defined DEM file as explained above.

SRTM1 product is 30m resolution DEM and SRTM3 is 90m. SRTM1 is much bigger (~10 times) and is usable for small areas. Mainly 90m SRTM3 is the right choice. Use parameter resolution_meters (60 meters by default) to interpolate the DEM to required resolution for the future processing and output.

The DEM grid is NetCDF file.

In [ ]:
if platform.system() == 'Darwin':
    # this command allows to download DEM for a large area but it requires GMT 6.1+ and it does not work on Google Colab
    sbas.download_dem(backend='GMT')
else:
    # for old Linux Debian and Ubuntu use default backend to download SRTM3 DEM for large areas (4°x4° and more)
    #sbas.download_dem(product='SRTM3')
    # for old Linux Debian and Ubuntu use default backend to download SRTM1 DEM for small areas (less than 4°x4°)
    sbas.download_dem(product='SRTM1')

### Static Plots

In [ ]:
plt.figure(figsize=(12,4), dpi=150)
dem = sbas.get_dem()[::4,::4]
dem.plot.imshow(cmap='gray', vmin=dem.min(), vmax=dem.max())
sbas.to_dataframe().plot(color='orange', alpha=0.2, ax=plt.gca())
plt.title('Sentinel1 Frame on DEM', fontsize=18)
plt.show()

### Interactive Plot

Be careful because interactive plots require more RAM to be visualized

The plots below do not work on Debian 10 and Python 3.7

In [ ]:
plots = None
if platform.system() == 'Darwin':
    title = 'Subswath Topo in Radar Coordinates'
    plots = sbas.get_dem()[::4,::4].hvplot(invert=True, cmap='gray', alpha=1, title=title) \
        * sbas.to_dataframe().reset_index().hvplot(color='date', alpha=0.2)
plots

## Align a Pair of Images

In [ ]:
sbas.stack_parallel()

## DEM in Radar Coordinates


In [ ]:
sbas.topo_ra_parallel()

### Load Grids

The grids are NetCDF files processing as xarray DataArrays.

In [ ]:
topo_ra = sbas.open_grid('topo_ra')

### Static Plot

In [ ]:
plt.figure(figsize=(12,4), dpi=300)
topo=topo_ra[::4,::4]
topo.plot.imshow(cmap='gray', vmin=topo.min(), vmax=topo.max())
plt.xlabel('Range', fontsize=16)
plt.ylabel('Azimuth', fontsize=16)
plt.title('Topography in Radar Coordinates', fontsize=18)
plt.show()

### Interactive Plot

Be careful because interactive plots require more RAM to be visualized

The plots below do not work on Debian 10 and Python 3.7

In [ ]:
plots = None
if platform.system() == 'Darwin':
    plot_opts = {'rasterize': True, 'xlabel':'Range', 'ylabel':'Azimuth', 'width':500, 'height':400}
    plots = topo_ra[::4,::4].hvplot(cmap='gray', title='Topography in Radar Coordinates', **plot_opts)
plots

## Interferogram

Define a single interferogram or a SBAS series. Make direct and reverse interferograms (from past to future or from future to past).

Decimation is useful to save disk space. Geocoding results are always produced on the provided DEM grid so the output grid and resolution are the same to the DEM. By this way, ascending and descending orbit results are always defined on the same grid by design. The internal processing cells is about 15 x 15 meters size and for default output 60m resolution (like to GMTSAR and GAMMA software) decimation 4x4 is reasonable. For the default wavelength=200 for Gaussian filter 1/4 of wavelength is approximately equal to ~60 meters and better resolution is mostly useless (while it can be used for small objects detection). For wavelength=400 meters use 90m or 120m DEM resolution with decimation 6x6 or 8x8 and for wavelength=100 meters use decimation 2x2.

The grids are NetCDF files processing as xarray DataArrays.

In [ ]:
pairs = [sbas.to_dataframe().index]

In [ ]:
# miss optional "func" argument when post-processing is not required
# define a postprocessing function for decimation as func=decimator
# for 30m DEM use decimation x=2, y=2 (is equal to GMTSAR rdec=8, adec=2) or 4x4 for 60m DEM
decimator = lambda dataarray: dataarray.coarsen({'y': 4, 'x': 4}, boundary='trim').median()

# default parameters: wavelength=200, psize=32, func=None (no postprocessing)
sbas.intf_parallel(pairs, func=decimator)

### Load Grids

The grids can be cropped automatically to drop empty areas around valid area (use crop_valid=True). On-the fly geocoding from radar to geographic coordinates is possible by geocode=True.

The grids are NetCDF files in radar coordinates processing as xarray DataArrays.  For a single interferogram processing convert the 3D stack to 2D grid using index [0].

In [ ]:
phasefilt = sbas.open_grids(pairs, 'phasefilt', geocode=True)[0]
corr = sbas.open_grids(pairs, 'corr', geocode=True)[0]

### Static Plot

In [ ]:
plt.figure(figsize=(12,4), dpi=300)
phasefilt.plot.imshow(vmin=-np.pi, vmax=np.pi, cmap='gist_rainbow_r')
plt.title('Phase, [rad]', fontsize=18)
plt.show()

### Interactive Plot

Be careful because interactive plots require more RAM to be visualized

The plots below do not work on Debian 10 and Python 3.7

In [ ]:
plots = None
if platform.system() == 'Darwin':
    opts_common = {'x':'lon', 'y':'lat', 'geo':True, 'width':800, 'height':600, 'colorbar':True, 'padding':0}
    opts1 = {'tiles':gstiles, 'alpha':0.4, 'cmap':'gist_rainbow_r', 'clim':(-np.pi, np.pi),
            'title': f'Sentinel-1 Interferogram [rad] Co-Seismic Pair {pairs[0][0]} - {pairs[0][1]}'}
    plots = phasefilt.hvplot(**{**opts_common, **opts1})
plots

## Landmask

Interferogram presents just a noise for water surfaces and unwrapping is meaningless long for these areas. Landmask allows to exclude water sufraces to produce better looking unwrapping results and much faster. Landmask in geographic coordinates is suitable to check it on the map while for unwrapping required landmask in radar coordinates.

In [ ]:
sbas.download_landmask()

### Load Grid

In [ ]:
landmask = sbas.get_landmask()

### Static Plot

In [ ]:
plt.figure(figsize=(12,4), dpi=300)
landmask.plot.imshow(vmin=0, vmax=1, cmap='gray')
plt.title('Landmask', fontsize=18)
plt.show()

### Interactive Plot

Be careful because interactive plots require more RAM to be visualized

The plots below do not work on Debian 10 and Python 3.7

In [ ]:
plots = None
if platform.system() == 'Darwin':
    opts_common = {'x':'lon', 'y':'lat', 'geo':True, 'width':800, 'height':600, 'colorbar':True, 'padding':0}
    opts1 = {'tiles':gstiles, 'alpha':0.4, 'cmap':'gray', 'clim':(0, 1),
            'title': f'Landmask'}
    plots = landmask.hvplot(**{**opts_common, **opts1})
plots

### Load Grid in Radar Coordinates

In [ ]:
landmask_ra = sbas.get_landmask(inverse_geocode=True)

In [ ]:
# create a custom mask using landmask or any other geographic coordinates grid
# landmask_ra = sbas.intf_ll2ra(landmask)

In [ ]:
# create Persistent Scatters map using all the stack correlations in radar coordinates
# landmask_ra = xr.where(sbas.open_grids(pairs, 'phasefilt').min('pair')>=CORRLIMIT, 1, 0)

### Static Plot

In [ ]:
plt.figure(figsize=(12,4), dpi=300)
landmask_ra.plot.imshow(vmin=0, vmax=1, cmap='gray')
plt.title('Landmask in Radar Coordinates', fontsize=18)
plt.show()

### Load Grids With Landmask

The grids can be cropped automatically to drop empty areas around valid area (use crop_valid=True). On-the fly geocoding from radar to geographic coordinates is possible by geocode=True.

The grids are NetCDF files in radar coordinates processing as xarray DataArrays.  For a single interferogram processing convert the 3D stack to 2D grid using index [0].

In [ ]:
phasefilt = sbas.open_grids(pairs, 'phasefilt', mask=landmask, geocode=True)[0]

### Static Plot

In [ ]:
plt.figure(figsize=(12,4), dpi=300)
phasefilt.plot.imshow(vmin=-np.pi, vmax=np.pi, cmap='gist_rainbow_r')
plt.title('Landmasked Phase, [rad]', fontsize=18)
plt.show()

### Centre of EO Research & Satellite Remote Sensing, Greece reported 5 fringes around the epicentre with deformation 5*2.8 cm = 14 cm (note: the same result. 6th fringe detected at the epicentre and it's hidden by 'star' marker on the geObservatory map)

https://www.linkedin.com/posts/beyond-centre-of-eo-research-855441172_δορυφορική-συμβολομετρική-απεικόνιση-του-activity-6849310648405430272-BG29/

In [ ]:
Image('S1AB_Crete_Earthquake_vs_geObsevatory.jpg', width="600")

## Unwrapping

Unwrapping process requires a lot of RAM and that's really RAM consuming when a lot of parallel proccesses running togeter. To limit the parallel processing tasks apply argument "n_jobs". The default value n_jobs=-1 means all the processor cores van be used. Also, use interferogram decimation above to produce smaller interferograms. And in addition a custom SNAPHU configuration can reduce RAM usage as explained below.

Attention: in case of crash on MacOS Apple Silicon run Jupyter as

`OBJC_DISABLE_INITIALIZE_FORK_SAFETY=YES no_proxy='*' jupyter notebook`

### Unwrapping Post-Processing

Define post-processing function to exclude low-coherence areas and maybe fill them by nearest neighbour interpolation.

In [ ]:
# define a post-processing function to crop low-coherence areas
cleaner = lambda corr, unwrap: xr.where(corr>=CORRLIMIT, unwrap, np.nan)

# define a post-processing function to crop and interpolate low-coherence areas
#cleaner = lambda corr, unwrap: sbas.nearest_grid(xr.where(corr>=CORRLIMIT, unwrap, np.nan))

In [ ]:
# "FileNotFoundError" SNAPHU error means the processing is not completed normally for some tiles
# try again with different SNAPHU parameters

# miss "func" argument or set it to None when post-processing is not required
# run only one unwrapping task at the time because the custom SNAPHU configuration already uses all the cores
sbas.unwrap_parallel(pairs, n_jobs=1, threshold=CORRLIMIT, mask=landmask_ra, func=cleaner)

### Load Grid

The grids can be cropped automatically to drop empty areas around valid area. Use `open_grids(..., crop_valid=True)` to enable the auto cropping feature. Argument geocode=True means on-the-fly geocoding from radar to geographic coordinates (all the grids saved on disk in radar coordinates plus geocoding matrices are generated for the fast geocoding). For a single interferogram processing convert the 3D stack to 2D grid using index [0].

In [ ]:
unwrap = sbas.open_grids(pairs, 'unwrap', geocode=True)[0]

### Static Plots

In [ ]:
plt.figure(figsize=(12,4), dpi=300)
unwrap.plot.imshow(cmap='jet')
plt.title('Landmasked Unwrapped Phase, [rad]', fontsize=18)
plt.show()

### Interactive Plot

Be careful because interactive plots require more RAM to be visualized

The plots below do not work on Debian 10 and Python 3.7

In [ ]:
plots = None
if platform.system() == 'Darwin':
    opts_common = {'x':'lon', 'y':'lat', 'geo':True, 'width':800, 'height':600, 'colorbar':True, 'padding':0}
    opts1 = {'tiles':gstiles, 'alpha':0.3, 'cmap':'turbo',
            'title': f'Sentinel-1 Landmasked Unwrapped Phase [rad] Co-Seismic Pair {pairs[0][0]} - {pairs[0][1]}'}
    plots = unwrap.hvplot(**{**opts_common, **opts1})
plots

## LOS Displacement

### Load and Calculate Grid

The grids can be post-processed using user-defined function "func". SBAS function `los_displacement_mm()` converts unwrapped phase values to LOS displacement in millimeters. For a single interferogramm processing convert the 3D stack to 2D grid using index [0].

In [ ]:
los_disp_mm = sbas.open_grids(pairs, 'unwrap', func=sbas.los_displacement_mm, geocode=True)[0]

### Static Plots

In [ ]:
plt.figure(figsize=(12,4), dpi=300)
zmin, zmax = np.nanquantile(los_disp_mm, [0.01, 0.99])
los_disp_mm.plot.imshow(vmin=zmin, vmax=zmax, cmap='jet')
plt.title('Landmasked LOS Displacement, [mm]', fontsize=18)
plt.show()

### Interactive Plot

Be careful because interactive plots require more RAM to be visualized

The plots below do not work on Debian 10 and Python 3.7

In [ ]:
plots = None
if platform.system() == 'Darwin':
    opts_common = {'x':'lon', 'y':'lat', 'geo':True, 'width':800, 'height':600, 'colorbar':True, 'padding':0}
    opts2 = {'tiles':gstiles, 'alpha':0.5, 'cmap':'turbo', 'clim':(zmin, zmax),
             'title': f'Sentinel-1 Landmasked LOS Displacement [mm] Co-Seismic Pair {pairs[0][0]} - {pairs[0][1]}'}
    plots = los_disp_mm.hvplot(**{**opts_common, **opts2})
plots

## Vertical and East-West Displacements

LOS displacement projections as vertical and east-west displacements can be calculated using incidence angle defined on the grid.

In [ ]:
incidence_angle = sbas.incidence_angle()

### Static Plot

In [ ]:
plt.figure(figsize=(12,4), dpi=300)
incidence_angle.plot.imshow(cmap='gray')
plt.title('Incidence Angle, [rad]', fontsize=18)
plt.show()

### Load and Calculate Grid

The grids can be post-processed using user-defined function "func". SBAS functions `vertical_displacement_mm()` and `eastwest_displacement_mm()` convert unwrapped phase values to vertical and East-West displacements in millimeters. For a single interferogramm processing convert the 3D stack to 2D grid using index [0].

In [ ]:
vert_disp_mm = sbas.open_grids(pairs, 'unwrap', func=sbas.vertical_displacement_mm, geocode=True)[0]
east_disp_mm = sbas.open_grids(pairs, 'unwrap', func=sbas.eastwest_displacement_mm, geocode=True)[0]

## Static Plots

In [ ]:
plt.figure(figsize=(12,4), dpi=300)
zmin, zmax = np.nanquantile(vert_disp_mm, [0.01, 0.99])
vert_disp_mm.plot.imshow(vmin=zmin, vmax=zmax, cmap='jet')
plt.title('Landmasked Vertical Displacement, [mm]', fontsize=18)
plt.show()

In [ ]:
plt.figure(figsize=(12,4), dpi=300)
zmin, zmax = np.nanquantile(east_disp_mm, [0.01, 0.99])
east_disp_mm.plot.imshow(vmin=zmin, vmax=zmax, cmap='jet')
plt.title('Landmasked East-West Displacement, [mm]', fontsize=18)
plt.show()

## Conclusion

For now you have the full control on interferometry processing and unwrapping and able to run it everywhere: on free of charge Google Colab instances, on local MacOS and Linux computers and on Amazon EC2 and Google Cloud VM and AI Notebook instances.